In [4]:
import sys
import os
import matplotlib.pyplot as plt
import torch
# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

from models.attention_unet.attention_unet import Attention_UNet
from src.models.data_management.cnn_formes import CNNFormes
from src.data_processing.dataset_loader import CoastData
from src.models.metrics import Metrics

from src.models.data_management.data_loader import DataLoaderManager

import cv2

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [6]:
artifact_path = os.path.abspath(os.path.join(os.getcwd(), "../../../artifacts/"))

num_classes = 3
metrics = {
    "train": Metrics(phase="train", num_classes=num_classes, average=None, use_margin=False),
    "validation": Metrics(phase="val", num_classes=num_classes, average=None, use_margin=False),
    "test": Metrics(phase="test", num_classes=num_classes, average=None, use_margin=False) # 'macro'
}

# Load the data to split it and save it to a dict
model = Attention_UNet(num_classes=3, experiment_name="test_experiments", use_mlflow=False)

model.load_model(os.path.abspath(os.path.join(artifact_path, "2025-03-06-22-30-23/models/best_model.pth")))

data_path = os.path.abspath(os.path.join(os.getcwd(), "../../../data/processed"))

coast_data = CoastData(data_path)
split = coast_data.split_data()

data = DataLoaderManager.load_data(split)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


CoastData: global - 1717 images
Coast: agrelo, Total size: 244
Coast: arenaldentem, Total size: 40
Coast: cadiz, Total size: 946
Coast: cies, Total size: 430
Coast: samarador, Total size: 57


In [11]:
for split in data:
    print(f"Split: {split}")
    for img_path, mask_path in zip(data[split]["images"], data[split]["masks"]):
        pred = model.predict_patch(img_path, combination="max")
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = torch.from_numpy(mask).float() 
        metrics[split].update_metrics(pred, mask)

    metrics[split].compute()
    print(metrics[split].get_last_epoch_info())

Split: train
train metrics: 
	train_accuracy: tensor([0.9984, 0.9891, 0.9922])
	train_f1_score: tensor([0.9987, 0.9906, 0.9904])
	train_precision: tensor([0.9989, 0.9921, 0.9887])
	train_recall: tensor([0.9984, 0.9891, 0.9922])
	train_confusion_matrix: 
		0.9984 0.0009 0.0007
		0.0004 0.9891 0.0105
		0.0004 0.0074 0.9922

Split: validation
val metrics: 
	val_accuracy: tensor([0.9987, 0.9821, 0.9830])
	val_f1_score: tensor([0.9988, 0.9826, 0.9825])
	val_precision: tensor([0.9989, 0.9830, 0.9819])
	val_recall: tensor([0.9987, 0.9821, 0.9830])
	val_confusion_matrix: 
		0.9987 0.0008 0.0005
		0.0006 0.9821 0.0174
		0.0003 0.0167 0.9830

Split: test
test metrics: 
	test_accuracy: tensor([0.9987, 0.9807, 0.9729])
	test_f1_score: tensor([0.9987, 0.9773, 0.9764])
	test_precision: tensor([0.9986, 0.9739, 0.9800])
	test_recall: tensor([0.9987, 0.9807, 0.9729])
	test_confusion_matrix: 
		0.9987 0.0009 0.0005
		0.0004 0.9807 0.0190
		0.0006 0.0264 0.9729

